In [306]:
#Importando las librerias necesarias 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import io
from scipy import stats
from statsmodels import robust
%matplotlib inline

In [307]:
Data1 = pd.read_csv('https://github.com/MinCiencia/Datos-COVID19/blob/master/output/producto29/Cuarentenas-Totales.csv?raw=true', sep=',')
#Leer la data de cuarentenas 

In [308]:
Data2 = pd.read_csv('https://github.com/MinCiencia/Datos-COVID19/blob/master/output/producto1/Covid-19.csv?raw=true', sep=',')
#Datos de casos por comuna

In [309]:
Data1.head()

,ID,Nombre,Estado,Alcance,Fecha de Inicio,Fecha de Término,Código CUT Comuna,Detalle,Superficie en m2,Perímetro en m,Region,n_REGION
0,1,Arica,Histórica,Área Urbana Completa,2020-04-16 22:00:00,2020-05-15 22:00:00,15101,Corresponde al Radio Urbano de la Ciudad de Ar...,28050418.80,48722.30001,15,R15
1,2,Santiago Norte,Histórica,Sector Específico,2020-04-13 05:00:00,2020-05-05 22:00:00,13101,Corresponde a la extensión de la cuarentena en...,19939218.57,21599.35134,13,R13
2,3,Puente Alto Poniente,Histórica,Sector Específico,2020-04-09 22:00:00,2020-05-08 22:00:00,13201,Corresponde al sector delimitado por la Av. Co...,42364268.21,30428.05152,13,R13
3,4,El Bosque,Histórica,Comuna completa,2020-04-16 22:00:00,2020-09-21 05:00:00,13105,Aplicada a la totalidad de la comuna. Se incor...,20622713.23,18788.87075,13,R13
4,5,San Bernardo Nororiente,Histórica,Sector Específico,2020-04-16 22:00:00,2020-05-08 22:00:00,13401,Corresponde al sector delimitado por los limit...,16326216.83,24929.14572,13,R13


In [310]:
Data2.head()

,Region,Codigo region,Comuna,Codigo comuna,Poblacion,2020-03-30,2020-04-01,2020-04-03,2020-04-06,2020-04-08,...,2021-09-10,2021-09-13,2021-09-17,2021-09-20,2021-09-24,2021-09-27,2021-10-01,2021-10-04,2021-10-08,Tasa
0,Arica y Parinacota,15,Arica,15101.0,247552.0,6.0,6.0,12.0,41.0,63.0,...,29763.0,29836.0,29929.0,30002.0,30128.0,30239.0,30376.0,30463.0,30536.0,12335.2
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,0.0,0.0,0.0,0.0,0.0,...,67.0,67.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,5677.2
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,0.0,0.0,0.0,0.0,0.0,...,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,10864.2
3,Arica y Parinacota,15,Putre,15201.0,2515.0,0.0,0.0,0.0,0.0,0.0,...,205.0,207.0,208.0,207.0,211.0,215.0,213.0,213.0,213.0,8469.2
4,Arica y Parinacota,15,Desconocido Arica y Parinacota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,158.0,163.0,161.0,163.0,165.0,165.0,167.0,170.0,175.0,NaN


In [311]:
NuevaData2 = Data2[['Region','Codigo region','Codigo comuna','Comuna','Poblacion']]
# Se busca eliminar las colunnas del data 2 que no son de utilidad por lo que se crea una NuevaData 2  
# que ya no contiene estas columnas

In [312]:
NuevaData2.head()

,Region,Codigo region,Codigo comuna,Comuna,Poblacion
0,Arica y Parinacota,15,15101.0,Arica,247552.0
1,Arica y Parinacota,15,15102.0,Camarones,1233.0
2,Arica y Parinacota,15,15202.0,General Lagos,810.0
3,Arica y Parinacota,15,15201.0,Putre,2515.0
4,Arica y Parinacota,15,NaN,Desconocido Arica y Parinacota,NaN


In [313]:
NuevaData2=Data2.dropna()
#elimino los Nan de la base de Data 2 

In [314]:
NuevaData2['Codigo comuna'].astype(int)
# Con este codigo se busca que el los datos del segundo dataframe sean iguales al primero para lograr combinarlos,
# para poder obtener un unico dataframe con la información sobre cuarentenas y poblaciones por comuna 

0      15101
1      15102
2      15202
3      15201
5       1107
       ...  
356    12101
357    12103
358    12104
359    12303
360    12402
Name: Codigo comuna, Length: 343, dtype: int32

In [315]:
Data1.drop(['Detalle', 'Superficie en m2','Perímetro en m','Alcance'], axis=1, inplace=True)

# Se eliminan las columnas que no son de interes en el primer data.

In [316]:
Data1.head()

,ID,Nombre,Estado,Fecha de Inicio,Fecha de Término,Código CUT Comuna,Region,n_REGION
0,1,Arica,Histórica,2020-04-16 22:00:00,2020-05-15 22:00:00,15101,15,R15
1,2,Santiago Norte,Histórica,2020-04-13 05:00:00,2020-05-05 22:00:00,13101,13,R13
2,3,Puente Alto Poniente,Histórica,2020-04-09 22:00:00,2020-05-08 22:00:00,13201,13,R13
3,4,El Bosque,Histórica,2020-04-16 22:00:00,2020-09-21 05:00:00,13105,13,R13
4,5,San Bernardo Nororiente,Histórica,2020-04-16 22:00:00,2020-05-08 22:00:00,13401,13,R13


In [317]:
NuevaData2.head()

,Region,Codigo region,Comuna,Codigo comuna,Poblacion,2020-03-30,2020-04-01,2020-04-03,2020-04-06,2020-04-08,...,2021-09-10,2021-09-13,2021-09-17,2021-09-20,2021-09-24,2021-09-27,2021-10-01,2021-10-04,2021-10-08,Tasa
0,Arica y Parinacota,15,Arica,15101.0,247552.0,6.0,6.0,12.0,41.0,63.0,...,29763.0,29836.0,29929.0,30002.0,30128.0,30239.0,30376.0,30463.0,30536.0,12335.2
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,0.0,0.0,0.0,0.0,0.0,...,67.0,67.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,5677.2
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,0.0,0.0,0.0,0.0,0.0,...,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,10864.2
3,Arica y Parinacota,15,Putre,15201.0,2515.0,0.0,0.0,0.0,0.0,0.0,...,205.0,207.0,208.0,207.0,211.0,215.0,213.0,213.0,213.0,8469.2
5,Tarapacá,1,Alto Hospicio,1107.0,129999.0,0.0,0.0,0.0,5.0,6.0,...,15812.0,15839.0,15866.0,15880.0,15902.0,15915.0,15942.0,15964.0,15997.0,12305.5
